In [1]:
import pandas as pd
import pyarrow
from pathlib import Path
import gc

In [2]:
df_sample = pd.read_parquet(
        'sample_50k.parquet',
        engine="pyarrow"
    )

In [3]:
diff = 30000 # difference of indexes in files

In [4]:
for n in range(30000, 270001, 30000): # numbers in files
    print(f'Processing chunk ruslawod{n}.parquet ...')
    
    df = pd.read_parquet(f'ruslawod{n}.parquet', engine="pyarrow")

    mask = (df_sample.index >= n - diff) & (df_sample.index < n) # mask to find correct index or specific file
    valid_global_indices = df_sample.index[mask]
    valid_local_indices = valid_global_indices - (n - diff) # It translates global indices (which are in df_sample.index) into local indices of the file ruslawod{n}.parquet

    df_filtered = df.loc[valid_local_indices].copy()

    df_filtered.to_parquet(f'ruslawod_sample{n}.parquet')

    del df
    del df_filtered
    del valid_global_indices
    del valid_local_indices
    gc.collect()

Processing chunk ruslawod30000.parquet ...
Processing chunk ruslawod60000.parquet ...
Processing chunk ruslawod90000.parquet ...
Processing chunk ruslawod120000.parquet ...
Processing chunk ruslawod150000.parquet ...
Processing chunk ruslawod180000.parquet ...
Processing chunk ruslawod210000.parquet ...
Processing chunk ruslawod240000.parquet ...
Processing chunk ruslawod270000.parquet ...


In [5]:
n = 300000

In [6]:
df = pd.read_parquet(f'ruslawod281233.parquet', engine="pyarrow")

mask = (df_sample.index >= n - diff) & (df_sample.index < n) # mask to find correct index or specific file
valid_global_indices = df_sample.index[mask]
valid_local_indices = valid_global_indices - (n - diff) # It translates global indices (which are in df_sample.index) into local indices of the file ruslawod{n}.parquet

df_filtered = df.loc[valid_local_indices]

df_filtered.to_parquet(f'ruslawod_sample281233.parquet')

In [7]:
df_filtered

,pravogovruNd,issuedByIPS,docdateIPS,docNumberIPS,doc_typeIPS,headingIPS,doc_author_normal_formIPS,signedIPS,statusIPS,actual_datetimeIPS,actual_datetime_humanIPS,is_widely_used,taggedtextIPS,textIPS,classifierByIPS,keywordsByIPS
1600,102011133,Постановление Кабинета Министров СССР,08.04.1991,149,Постановление,О назначении т. Маковеева В. Г. заместителем П...,Кабинет Министров СССР,"Павлов В., ПРОСТЯКОВ И.",Действует без изменений,1710561008.5163362,Sat Mar 16 06:50:08 2024,1,None,\n КАБИНЕТ МИНИСТРОВ СССР \n ПОСТАНОВЛЕНИЕ \n...,None,None
3836,102021124,Распоряжение Совета Министров - Правительства ...,20.01.1993,79-р,Распоряжение,О заместителях Председателя Государственного к...,Совет Министров - Правительство Российской Фед...,Черномырдин В.,Действует без изменений,1710557019.790117,Sat Mar 16 05:43:39 2024,0,None,СОВЕТ МИНИСТРОВ - ПРАВИТЕЛЬСТВО \n РОССИЙСКОЙ...,210.010.010.200.030$*Государственный комитет Р...,"АРХИТЕКТУРА, ГОСУДАРСТВЕННЫЙ, ЗАМЕСТИТЕЛЬ, КОМ..."
7568,102071250,Федеральный закон,28.05.2001,62-ФЗ,Федеральный закон,"О ратификации Конвенции об отмывании, выявлени...",Государственная Дума,Путин В.,Действует без изменений,1710296215.051094,Wed Mar 13 05:16:55 2024,0,None,\nРОССИЙСКАЯ ФЕДЕРАЦИЯ\n \nФЕДЕРАЛЬНЫЙ ЗАКОН\...,020.010.020.020.000$Компетенция $200.070.020.0...,"ГЕНЕРАЛЬНЫЙ, ГРАЖДАНСКИЙ, ДЕЛО, ДЕЯТЕЛЬНОСТЬ, ..."
449,102153924,Распоряжение Правительства Российской Федерации,06.02.2012,141-р,Распоряжение,(О внесении изменений в состав организационног...,Правительство Российской Федерации,Путин В.,Утратил силу,1710484588.566326,Fri Mar 15 09:36:28 2024,0,None,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \n РАСПОРЯ...,010.070.020.040.450$Тульская область $010.140....,"ИЗМЕНЕНИЕ, КОМИТЕТ, ОРГАНИЗАЦИОННЫЙ, ПРОИЗВОДС..."
10458,102034613,Указ Президента Российской Федерации,10.03.1995,254,Указ,О выходе из гражданства Российской Федерации,Президент Российской Федерации,Ельцин Б.,Действует без изменений,1710551664.736981,Sat Mar 16 04:14:24 2024,1,None,\nУКАЗ\n \nПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ\n ...,210.040.020.000.000$Выход из гражданства,"ВЫХОД, ГРАЖДАНСТВО"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,102011426,Постановление Кабинета Министров СССР,14.05.1991,242,Постановление,О Государственном агентстве СССР по авторским ...,Кабинет Министров СССР,"Павлов В., ПРОСТЯКОВ И.",Действует без изменений,1710561201.507785,Sat Mar 16 06:53:21 2024,1,None,\n КАБИНЕТ МИНИСТРОВ СССР \n ПОСТАНОВЛЕНИЕ \n...,None,None
4707,102088051,Указ Президента Российской Федерации,28.07.2004,974,Указ,О назначении Лысенко М.Н. Чрезвычайным и Полно...,Президент Российской Федерации,Путин В.,Действует без изменений,1710321219.2793703,Wed Mar 13 12:13:39 2024,0,None,\nУКАЗ\n \nПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ\n ...,200.010.020.060.030$Самоа (Независимое Государ...,"НАЗНАЧЕНИЕ, ПОСОЛ"
8084,102381818,Указ Президента Российской Федерации,14.11.2015,559,Указ,О внесении изменений в состав Совета при Прези...,Президент Российской Федерации,Путин В.,Утратил силу,1710435902.85313,Thu Mar 14 20:05:02 2024,0,None,\nУКАЗ\n \nПРЕЗИДЕНТА РОССИЙСКОЙ ФЕДЕРАЦИИ\n ...,010.090.050.010.160$Совет при Президенте Росси...,"ИЗМЕНЕНИЕ, КУЛЬТУРА, ПРЕЗИДЕНТ, СОВЕТ, СОСТАВ,..."
9872,102010491,Постановление Президиума Верховного Совета РСФСР,30.01.1991,557-I,Постановление,О преобразовании Государственного комитета РСФ...,Президиум Верховного Совета РСФСР,Ельцин Б.,Действует без изменений,1710561379.4391506,Sat Mar 16 06:56:19 2024,1,None,\n ПОСТАНОВЛЕНИЕ ПРЕЗИДИУМА ВЕРХОВНОГО СОВЕТА...,020.010.040.100.303$*Государственный комитет Р...,"БЕЗОПАСНОСТЬ, ГОСУДАРСТВЕННЫЙ, КОМИТЕТ, ОБОРОН..."


Combine all samples with text low data to one dataset of 50k

In [ ]:
DATA_DIR = Path(".")     
PATTERN = "ruslawod_sample*.parquet" 

dfs = []
for f in DATA_DIR.glob(PATTERN):
    df = pd.read_parquet(
        f,
        engine="pyarrow"
    )
    dfs.append(df)

In [18]:
df_full = pd.concat(dfs, ignore_index=True)

In [ ]:
df_full.to_parquet('data.parquet')